In [1]:
#dependencies
import re
from bs4 import BeautifulSoup 
from splinter import Browser
import requests
import pandas as pd
from selenium import webdriver 
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service as BraveService
from webdriver_manager.core.utils import ChromeType
import json
import time
from tqdm.notebook import tqdm 

<h4> Start cells here if partial FIPS already available in {state}_fips_scraped file

In [2]:
fips_check_scraped = pd.read_csv("../../../data/state_data/geo/geo_fips/23/TN_fips_scraped.csv") 
fips_check_scraped  = fips_check_scraped.loc[:, ~fips_check_scraped.columns.str.contains('^Unnamed')] 
fips_check_scraped.head() 

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,1057357305,TN,"5141 VIRGINIA WAY, SUITE 350, BRENTWOOD, TN",36.031762,-86.806264,4.718705e+14
1,2026197100,TN,"2785 OLD JACKSON RD, HENDERSON, TN",35.475051,-88.689976,4.702397e+14
2,2038767102,TN,"813 RIDGE LAKE BLVD, MEMPHIS, TN",35.109352,-89.868530,4.715702e+14
3,4079487205,TN,"7945 WOLF RIVER PARKWAY, GERMANTOWN, TN",35.110824,-89.798485,4.715702e+14
4,4290727102,TN,"216 Centerview Drive, BRENTWOOD, TN",36.028875,-86.787250,4.718705e+14


In [3]:
#Latitude, Longitude, not included
fips_check_scraped_notna = fips_check_scraped[fips_check_scraped["Lat"].notna()] # check to make sure no nulls in coords
fips_check_scraped_notna.head() 

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,1057357305,TN,"5141 VIRGINIA WAY, SUITE 350, BRENTWOOD, TN",36.031762,-86.806264,4.718705e+14
1,2026197100,TN,"2785 OLD JACKSON RD, HENDERSON, TN",35.475051,-88.689976,4.702397e+14
2,2038767102,TN,"813 RIDGE LAKE BLVD, MEMPHIS, TN",35.109352,-89.868530,4.715702e+14
3,4079487205,TN,"7945 WOLF RIVER PARKWAY, GERMANTOWN, TN",35.110824,-89.798485,4.715702e+14
4,4290727102,TN,"216 Centerview Drive, BRENTWOOD, TN",36.028875,-86.787250,4.718705e+14


In [4]:
#FIPS not included, of Lat, Long available
fips_check_scraped_null = fips_check_scraped_notna[fips_check_scraped_notna["FIPS_z"].isnull()] # check to make sure no nulls in coords
fips_check_scraped_null #28093

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
109,8820317006,TN,"207 Mockingbird Lane, JOHNSON CITY, TN",36.335528,-82.375058,NaN
658,5160427004,TN,"227 US HIGHWAY 45, HUMBOLDT, TN",43.772419,-88.446956,NaN
846,4377517003,TN,"4533 HIGHWAY 11, BLUFF CITY, TN",31.223458,-89.355297,NaN
1021,8952968403,TN,"1323 US Highway 45 N, Henderson, TN",42.222749,-87.969957,NaN
1088,5255147006,TN,"1323 US Hwy 45 N, HENDERSON, TN",42.222749,-87.969957,NaN
2185,7268808302,TN,"126 Highway 51 S, Covington, TN",39.934096,-88.954628,NaN
2186,7432757002,TN,"126 HIGHWAY 51, COVINGTON, TN",39.934096,-88.954628,NaN
2705,9280087110,TN,"3495 US HIGHWAY 45 S, PINSON, TN",36.710172,-88.682203,NaN
2839,9264117008,TN,"PO BOX 331821, MURFREESBORO, TN",35.845621,-86.390270,NaN
3063,4967028604,TN,"1992 Highway 51 S, Covington, TN",37.497817,-89.231936,NaN


<h4>FIPS Script

In [5]:
#if FIPS_z file 
lat = fips_check_scraped_null["Lat"].astype(float).astype(str).to_list()
long = fips_check_scraped_null["Long"].astype(float).astype(str).to_list()

In [6]:
#before running script, check for any NaN within Lat, Long fields. 
def sleep():
    time.sleep(1)
option = webdriver.ChromeOptions()
option.binary_location = "/Applications/Brave Browser.app/Contents/MacOS/Brave Browser"
driver = Browser('chrome', options=option, headless=False)    
url = 'https://geocoding.geo.census.gov/geocoder/geographies/coordinates?form'
driver.visit(url)

not_found = []
geo_id = []

i=0
for i in tqdm(range(len(lat))):
    #while i < len(lat):
        driver.find_by_id('geoCoordX').clear()
        driver.find_by_id('geoCoordY').clear()
        driver.find_by_id('geoCoordX').fill(long[i])
        driver.find_by_id('geoCoordY').fill(lat[i])
        driver.find_by_id('geoCoordsSubmit').click()
        #sleep()
        #html = driver.html
        soup = BeautifulSoup(driver.html, 'html.parser')


        results = soup.find_all('div', id='geoCoordsJsonOutput')
        for result in results:
            result_text = result.text

            result_digits = re.findall('\d+', result_text)
            startswith = [s for s in result_digits if s.startswith('47')] #TN
            fips_elements =[element for element in startswith if len(element)>=15]
            
            try:
                #print(f"{i},{lat[i]},{long[i]} is {fips_elements}")
                geo_id.append(fips_elements)
            except (KeyError, IndexError):
                #print(f"{i},{lat[i]},{long[i]} Not Found")
                not_found.append(i)
                geo_id.append(None)  
            except (TimeoutException):
                driver.navigate.refresh()
    
        #sleep()
        i=i+1

  0%|          | 0/29 [00:00<?, ?it/s]

coordinates wtih more than one return:
385,37.514070000000004,-77.35472 is ['514070000000004', '510872015041003'] 
1178,37.516454700000004,-77.8100575 is ['516454700000004', '511455001022014'] 
2660,37.519462299999994,-77.340384 is ['519462299999994', '510872014061012']
second option reference FIPS code, first option references LatCoord, update fields in DataFrame

https://iqss.github.io/dss-webscrape/filling-in-web-forms.html

In [7]:
geo_id_list =geo_id 

Length Check

In [8]:
print(len(geo_id_list))

29


Matching FIPS to DataFrame

In [9]:
fips_fill = fips_check_scraped_null

In [10]:
fips_fill["FIPS_z"] = geo_id_list
fips_fill

/var/folders/vy/qdp8xcfj4s3dd5c6d1xp_y3w0000gn/T/ipykernel_11298/597263667.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fips_fill["FIPS_z"] = geo_id_list


,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
109,8820317006,TN,"207 Mockingbird Lane, JOHNSON CITY, TN",36.335528,-82.375058,[471790604012017]
658,5160427004,TN,"227 US HIGHWAY 45, HUMBOLDT, TN",43.772419,-88.446956,[]
846,4377517003,TN,"4533 HIGHWAY 11, BLUFF CITY, TN",31.223458,-89.355297,[]
1021,8952968403,TN,"1323 US Highway 45 N, Henderson, TN",42.222749,-87.969957,[]
1088,5255147006,TN,"1323 US Hwy 45 N, HENDERSON, TN",42.222749,-87.969957,[]
2185,7268808302,TN,"126 Highway 51 S, Covington, TN",39.934096,-88.954628,[]
2186,7432757002,TN,"126 HIGHWAY 51, COVINGTON, TN",39.934096,-88.954628,[]
2705,9280087110,TN,"3495 US HIGHWAY 45 S, PINSON, TN",36.710172,-88.682203,[]
2839,9264117008,TN,"PO BOX 331821, MURFREESBORO, TN",35.845621,-86.390270,[471490416021025]
3063,4967028604,TN,"1992 Highway 51 S, Covington, TN",37.497817,-89.231936,[]


Remove brackets

In [11]:
fips_fill['FIPS_z'] = fips_fill['FIPS_z'].str[0]
fips_fill

/var/folders/vy/qdp8xcfj4s3dd5c6d1xp_y3w0000gn/T/ipykernel_11298/1219885402.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fips_fill['FIPS_z'] = fips_fill['FIPS_z'].str[0]


,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
109,8820317006,TN,"207 Mockingbird Lane, JOHNSON CITY, TN",36.335528,-82.375058,471790604012017
658,5160427004,TN,"227 US HIGHWAY 45, HUMBOLDT, TN",43.772419,-88.446956,NaN
846,4377517003,TN,"4533 HIGHWAY 11, BLUFF CITY, TN",31.223458,-89.355297,NaN
1021,8952968403,TN,"1323 US Highway 45 N, Henderson, TN",42.222749,-87.969957,NaN
1088,5255147006,TN,"1323 US Hwy 45 N, HENDERSON, TN",42.222749,-87.969957,NaN
2185,7268808302,TN,"126 Highway 51 S, Covington, TN",39.934096,-88.954628,NaN
2186,7432757002,TN,"126 HIGHWAY 51, COVINGTON, TN",39.934096,-88.954628,NaN
2705,9280087110,TN,"3495 US HIGHWAY 45 S, PINSON, TN",36.710172,-88.682203,NaN
2839,9264117008,TN,"PO BOX 331821, MURFREESBORO, TN",35.845621,-86.390270,471490416021025
3063,4967028604,TN,"1992 Highway 51 S, Covington, TN",37.497817,-89.231936,NaN


In [12]:
fips_fill.loc[fips_fill["FIPS_z"].isnull()] #check #13

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
658,5160427004,TN,"227 US HIGHWAY 45, HUMBOLDT, TN",43.772419,-88.446956,NaN
846,4377517003,TN,"4533 HIGHWAY 11, BLUFF CITY, TN",31.223458,-89.355297,NaN
1021,8952968403,TN,"1323 US Highway 45 N, Henderson, TN",42.222749,-87.969957,NaN
1088,5255147006,TN,"1323 US Hwy 45 N, HENDERSON, TN",42.222749,-87.969957,NaN
2185,7268808302,TN,"126 Highway 51 S, Covington, TN",39.934096,-88.954628,NaN
2186,7432757002,TN,"126 HIGHWAY 51, COVINGTON, TN",39.934096,-88.954628,NaN
2705,9280087110,TN,"3495 US HIGHWAY 45 S, PINSON, TN",36.710172,-88.682203,NaN
3063,4967028604,TN,"1992 Highway 51 S, Covington, TN",37.497817,-89.231936,NaN
4261,4202617704,TN,"27603 Commerce Oaks Drive, Oaks Ridge North, TN",30.159098,-95.435116,NaN
4753,5309117102,TN,"300 WOODLAWN RD, SEQUATCHIE, TN",39.347972,-76.634589,NaN


Parse for Merge

In [13]:
cols= ["LoanNumber", "FIPS_z"]
fips_fill_parsed = fips_fill[cols]
fips_fill_parsed

,LoanNumber,FIPS_z
109,8820317006,471790604012017
658,5160427004,NaN
846,4377517003,NaN
1021,8952968403,NaN
1088,5255147006,NaN
2185,7268808302,NaN
2186,7432757002,NaN
2705,9280087110,NaN
2839,9264117008,471490416021025
3063,4967028604,NaN


Read in base data

In [14]:

#if from FIPS file
fips_check = pd.read_csv("../../../data/state_data/geo/geo_fips/23/TN_fips_scraped.csv") 
#fips_check = fips_check.drop(["Unnamed: 0.1", "Unnamed: 0.1.1", "Unnamed: 0"], axis =1) 
fips_check.head()

,Unnamed: 0,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,792113,1057357305,TN,"5141 VIRGINIA WAY, SUITE 350, BRENTWOOD, TN",36.031762,-86.806264,4.718705e+14
1,792114,2026197100,TN,"2785 OLD JACKSON RD, HENDERSON, TN",35.475051,-88.689976,4.702397e+14
2,792115,2038767102,TN,"813 RIDGE LAKE BLVD, MEMPHIS, TN",35.109352,-89.868530,4.715702e+14
3,792116,4079487205,TN,"7945 WOLF RIVER PARKWAY, GERMANTOWN, TN",35.110824,-89.798485,4.715702e+14
4,792117,4290727102,TN,"216 Centerview Drive, BRENTWOOD, TN",36.028875,-86.787250,4.718705e+14


Merge with MainFrame

In [15]:
fips_merge = fips_check.merge(fips_fill_parsed, on="LoanNumber", how="left")
fips_merge

,Unnamed: 0,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z_x,FIPS_z_y
0,792113,1057357305,TN,"5141 VIRGINIA WAY, SUITE 350, BRENTWOOD, TN",36.031762,-86.806264,4.718705e+14,NaN
1,792114,2026197100,TN,"2785 OLD JACKSON RD, HENDERSON, TN",35.475051,-88.689976,4.702397e+14,NaN
2,792115,2038767102,TN,"813 RIDGE LAKE BLVD, MEMPHIS, TN",35.109352,-89.868530,4.715702e+14,NaN
3,792116,4079487205,TN,"7945 WOLF RIVER PARKWAY, GERMANTOWN, TN",35.110824,-89.798485,4.715702e+14,NaN
4,792117,4290727102,TN,"216 Centerview Drive, BRENTWOOD, TN",36.028875,-86.787250,4.718705e+14,NaN
...,...,...,...,...,...,...,...,...
15859,807972,9750047104,TN,"7594 John Bragg Highway, Murfreesboro, TN",35.823325,-86.254171,4.714904e+14,NaN
15860,807973,9848117110,TN,"9019 OVERLOOK BLVD, BRENTWOOD, TN",36.030708,-86.777887,4.718705e+14,NaN
15861,807974,9879197202,TN,"11450 Shaw Ferry Rd, Lenoir City, TN",35.828048,-84.272926,4.710506e+14,NaN
15862,807975,9893378603,TN,"4601 Nolensville Pike, Nashville, TN",36.069519,-86.722029,4.703702e+14,NaN


<b>Fill FIPS Columns - if FIPS file

In [16]:
fips_merge["FIPS_z"] = fips_merge["FIPS_z_y"].fillna(fips_merge["FIPS_z_x"])
fips_merge.head()

,Unnamed: 0,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z_x,FIPS_z_y,FIPS_z
0,792113,1057357305,TN,"5141 VIRGINIA WAY, SUITE 350, BRENTWOOD, TN",36.031762,-86.806264,4.718705e+14,NaN,471870503042028.0
1,792114,2026197100,TN,"2785 OLD JACKSON RD, HENDERSON, TN",35.475051,-88.689976,4.702397e+14,NaN,470239703022016.0
2,792115,2038767102,TN,"813 RIDGE LAKE BLVD, MEMPHIS, TN",35.109352,-89.868530,4.715702e+14,NaN,471570213111020.0
3,792116,4079487205,TN,"7945 WOLF RIVER PARKWAY, GERMANTOWN, TN",35.110824,-89.798485,4.715702e+14,NaN,471570213511003.0
4,792117,4290727102,TN,"216 Centerview Drive, BRENTWOOD, TN",36.028875,-86.787250,4.718705e+14,NaN,471870503042050.0


Drop Extra FIPS Columns

In [17]:
fips_merge_drop = fips_merge.drop(["FIPS_z_x", "FIPS_z_y"], axis=1)
fips_merge_drop = fips_merge_drop.drop(["Unnamed: 0"], axis=1)
fips_merge_drop

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
0,1057357305,TN,"5141 VIRGINIA WAY, SUITE 350, BRENTWOOD, TN",36.031762,-86.806264,471870503042028.0
1,2026197100,TN,"2785 OLD JACKSON RD, HENDERSON, TN",35.475051,-88.689976,470239703022016.0
2,2038767102,TN,"813 RIDGE LAKE BLVD, MEMPHIS, TN",35.109352,-89.868530,471570213111020.0
3,4079487205,TN,"7945 WOLF RIVER PARKWAY, GERMANTOWN, TN",35.110824,-89.798485,471570213511003.0
4,4290727102,TN,"216 Centerview Drive, BRENTWOOD, TN",36.028875,-86.787250,471870503042050.0
...,...,...,...,...,...,...
15859,9750047104,TN,"7594 John Bragg Highway, Murfreesboro, TN",35.823325,-86.254171,471490406004001.0
15860,9848117110,TN,"9019 OVERLOOK BLVD, BRENTWOOD, TN",36.030708,-86.777887,471870502101006.0
15861,9879197202,TN,"11450 Shaw Ferry Rd, Lenoir City, TN",35.828048,-84.272926,471050603011022.0
15862,9893378603,TN,"4601 Nolensville Pike, Nashville, TN",36.069519,-86.722029,470370189042003.0


In [18]:
fips_merge_drop.loc[fips_merge_drop["FIPS_z"].isnull()] #13

,LoanNumber,BorrowerState,full_add,Lat,Long,FIPS_z
658,5160427004,TN,"227 US HIGHWAY 45, HUMBOLDT, TN",43.772419,-88.446956,NaN
846,4377517003,TN,"4533 HIGHWAY 11, BLUFF CITY, TN",31.223458,-89.355297,NaN
1021,8952968403,TN,"1323 US Highway 45 N, Henderson, TN",42.222749,-87.969957,NaN
1088,5255147006,TN,"1323 US Hwy 45 N, HENDERSON, TN",42.222749,-87.969957,NaN
2185,7268808302,TN,"126 Highway 51 S, Covington, TN",39.934096,-88.954628,NaN
2186,7432757002,TN,"126 HIGHWAY 51, COVINGTON, TN",39.934096,-88.954628,NaN
2705,9280087110,TN,"3495 US HIGHWAY 45 S, PINSON, TN",36.710172,-88.682203,NaN
3063,4967028604,TN,"1992 Highway 51 S, Covington, TN",37.497817,-89.231936,NaN
4261,4202617704,TN,"27603 Commerce Oaks Drive, Oaks Ridge North, TN",30.159098,-95.435116,NaN
4753,5309117102,TN,"300 WOODLAWN RD, SEQUATCHIE, TN",39.347972,-76.634589,NaN


Convert to File

In [19]:
fips_TN = fips_merge_drop

In [20]:
fips_TN.to_csv("../../../data/state_data/geo/geo_fips/23/TN_fips_scraped.csv") 